In [229]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.tokenize import word_tokenize
import nltk
import pyarabic.araby as araby
from pyarabic.araby import strip_tashkeel
import qalsadi.lemmatizer 
import qalsadi.analex as qa
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from farasa.pos import FarasaPOSTagger 
from farasa.ner import FarasaNamedEntityRecognizer 
from farasa.diacratizer import FarasaDiacritizer 
from farasa.segmenter import FarasaSegmenter 
from farasa.stemmer import FarasaStemmer

import keras
from  diacritization_evaluation import util

In [230]:
words = []
sentences = []
sentences_with_tashkeel = []
with open('./Dataset/WordsWithoutTashkeel.txt', 'r', encoding='utf-8') as output_file:
    for word in output_file:
        words.append(word.strip())

with open('./Dataset/SentencesWithoutTashkeel.txt', 'r', encoding='utf-8') as output_file:
    for sentence in output_file:
        sentences.append(sentence.strip())

with open('./Dataset/sentences.txt', 'r', encoding='utf-8') as output_file:
    for sentence in output_file:
        sentences_with_tashkeel.append(sentence.strip())

print(words[0:10])
print(sentences[0])
print(sentences_with_tashkeel[0])


['قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزركشي', 'ابن', 'عرفة']
قوله أو قطع الأول يده إلخ قال الزركشي ابن عرفة قوله بلفظ يقتضيه كإنكار غير حديث بالإسلام وجوب ما علم وجوبه من الدين ضرورة كإلقاء مصحف بقذر وشد زنار ابن عرفة قول ابن شاس أو بفعل يتضمنه هو كلبس الزنار وإلقاء المصحف في صريح النجاسة والسجود للصنم ونحو ذلك وسحر محمد قول مالك وأصحابه أن الساحر كافر بالله تعالى قال مالك هو كالزنديق إذا عمل السحر بنفسه قتل ولم يستتب
قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ابْنُ عَرَفَةَ قَوْلُ ابْنِ شَاسٍ أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ وَسِحْرٍ مُحَمَّدٌ قَوْلُ مَالِكٍ وَأَصْحَابِهِ أَنَّ السَّاحِرَ كَافِرٌ بِاَللَّهِ تَعَالَى قَالَ مَالِكٌ هُوَ كَالزِّنْدِيقِ إذَا 

In [231]:
print(sentences_with_tashkeel[0])

قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ابْنُ عَرَفَةَ قَوْلُ ابْنِ شَاسٍ أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ وَسِحْرٍ مُحَمَّدٌ قَوْلُ مَالِكٍ وَأَصْحَابِهِ أَنَّ السَّاحِرَ كَافِرٌ بِاَللَّهِ تَعَالَى قَالَ مَالِكٌ هُوَ كَالزِّنْدِيقِ إذَا عَمِلَ السِّحْرَ بِنَفْسِهِ قُتِلَ وَلَمْ يُسْتَتَبْ


Creating a word-based tokenizer

In [232]:
# Create a tokenizer
words_tokenizer = Tokenizer()

# Fit the tokenizer on the list of words (treat each word as a separate "sentence")
words_tokenizer.fit_on_texts(sentences)

# Get the word index
word_index = words_tokenizer.word_index

# Tokenize the words
word_sequences = words_tokenizer.texts_to_sequences(sentences)

Testing the tokenizer on a sample sentence

In [233]:
# Create a sentence tokenizer
print((sentences[0].split(" "))) #This way works and the one used below as well
print(word_index["قوله"])
print(words_tokenizer.texts_to_sequences([sentences[0]]))

['قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزركشي', 'ابن', 'عرفة', 'قوله', 'بلفظ', 'يقتضيه', 'كإنكار', 'غير', 'حديث', 'بالإسلام', 'وجوب', 'ما', 'علم', 'وجوبه', 'من', 'الدين', 'ضرورة', 'كإلقاء', 'مصحف', 'بقذر', 'وشد', 'زنار', 'ابن', 'عرفة', 'قول', 'ابن', 'شاس', 'أو', 'بفعل', 'يتضمنه', 'هو', 'كلبس', 'الزنار', 'وإلقاء', 'المصحف', 'في', 'صريح', 'النجاسة', 'والسجود', 'للصنم', 'ونحو', 'ذلك', 'وسحر', 'محمد', 'قول', 'مالك', 'وأصحابه', 'أن', 'الساحر', 'كافر', 'بالله', 'تعالى', 'قال', 'مالك', 'هو', 'كالزنديق', 'إذا', 'عمل', 'السحر', 'بنفسه', 'قتل', 'ولم', 'يستتب']
8
[[8, 4, 273, 91, 189, 47, 14, 901, 37, 458, 8, 1184, 3641, 41299, 39, 249, 4154, 475, 12, 163, 1738, 3, 158, 1175, 27872, 6099, 41300, 5827, 41301, 37, 458, 63, 37, 2652, 4, 1362, 24279, 43, 17883, 57670, 41302, 2961, 2, 1430, 2148, 9577, 57671, 1189, 18, 21681, 86, 63, 94, 3359, 6, 16422, 1190, 1185, 108, 14, 94, 43, 27873, 28, 520, 8288, 415, 264, 51, 57672]]


Creating a character-based tokenizer

In [234]:
with open('arabic_letters.pickle', 'rb') as file:
    arabic_letters = pickle.load(file)

with open('diacritic2id.pickle', 'rb') as file:
    diacritics = pickle.load(file)

letters_and_diacritics = list(arabic_letters) + list(diacritics.keys())
print(letters_and_diacritics)

['أ', 'ث', 'ه', 'ض', 'ب', 'ي', 'و', 'ا', 'ئ', 'ك', 'ذ', 'ص', 'ظ', 'ف', 'آ', 'س', 'د', 'ح', 'ق', 'ى', 'ؤ', 'م', 'ز', 'ط', 'ة', 'ل', 'خ', 'ت', 'ر', 'ش', 'ع', 'غ', 'إ', 'ن', 'ج', 'ء', 'َ', 'ً', 'ُ', 'ٌ', 'ِ', 'ٍ', 'ْ', 'ّ', 'َّ', 'ًّ', 'ُّ', 'ٌّ', 'ِّ', 'ٍّ', '']


In [235]:
sentences_new = []
with open('./Dataset/sentences_new_approach.txt', 'r', encoding='utf-8') as output_file:
    for sentence in output_file:
        sentences_new.append(sentence.strip())

char_tokenizer = Tokenizer(char_level=True, oov_token='UNK')
char_tokenizer.fit_on_texts(sentences_new)
char_index = char_tokenizer.word_index
char_sequences = char_tokenizer.texts_to_sequences(sentences_new)
print(char_sequences[0:5])
print(char_index)

[[20, 2, 11, 5, 6, 8, 13, 8, 3, 18, 2, 11, 5, 3, 20, 2, 41, 2, 16, 2, 3, 7, 6, 5, 18, 2, 11, 17, 6, 8, 3, 10, 2, 22, 2, 13, 8, 3, 28, 6, 2, 34, 5, 3, 20, 2, 7, 6, 2, 3, 7, 6, 42, 17, 15, 5, 23, 2, 37, 4, 10, 43, 3, 7, 14, 5, 12, 8, 3, 16, 2, 15, 2, 19, 2, 26, 2, 3, 20, 2, 11, 5, 6, 8, 13, 8, 3, 14, 4, 6, 2, 19, 5, 47, 31, 3, 10, 2, 20, 5, 21, 2, 38, 4, 10, 13, 3, 23, 2, 28, 4, 12, 5, 23, 2, 7, 15, 4, 3, 44, 2, 10, 5, 15, 4, 3, 24, 2, 22, 4, 10, 35, 31, 3, 14, 4, 7, 6, 5, 28, 4, 25, 5, 6, 2, 7, 9, 4, 3, 11, 8, 27, 8, 11, 14, 2, 3, 9, 2, 7, 3, 16, 8, 6, 4, 9, 2, 3, 11, 8, 27, 8, 11, 14, 8, 13, 8, 3, 9, 4, 12, 5, 3, 7, 6, 22, 36, 10, 12, 4, 3, 38, 2, 15, 8, 11, 15, 2, 26, 39, 3, 23, 2, 28, 4, 6, 5, 20, 2, 7, 45, 4, 3, 9, 8, 30, 5, 24, 2, 19, 31, 3, 14, 4, 20, 2, 29, 2, 15, 31, 3, 11, 2, 37, 2, 22, 36, 3, 42, 8, 12, 17, 7, 15, 31, 3, 7, 14, 5, 12, 8, 3, 16, 2, 15, 2, 19, 2, 26, 2, 3, 20, 2, 11, 5, 6, 8, 3, 7, 14, 5, 12, 4, 3, 37, 2, 7, 25, 31, 3, 18, 2, 11, 5, 3, 14, 4, 19, 4, 16, 5, 6, 31

Adding tests for the character-based tokenizer

In [236]:
print(char_index)
print(char_tokenizer.texts_to_sequences(sentences_new[12].split(" ")))


{'UNK': 1, 'َ': 2, ' ': 3, 'ِ': 4, 'ْ': 5, 'ل': 6, 'ا': 7, 'ُ': 8, 'م': 9, 'ي': 10, 'و': 11, 'ن': 12, 'ه': 13, 'ب': 14, 'ر': 15, 'ع': 16, 'd': 17, 'أ': 18, 'ف': 19, 'ق': 20, 'ت': 21, 'د': 22, 'ك': 23, 'ح': 24, 'س': 25, 'ة': 26, 'ج': 27, 'إ': 28, 'ذ': 29, 'ص': 30, 'ٍ': 31, '،': 32, 'ى': 33, 'خ': 34, 'ث': 35, 'f': 36, 'ش': 37, 'ض': 38, 'ً': 39, 'ٌ': 40, 'ط': 41, 'ز': 42, 'e': 43, 'غ': 44, 'ء': 45, 'ئ': 46, 'ظ': 47, 'آ': 48, 'ؤ': 49, 'ّ': 50, 'c': 51, 'b': 52, 'a': 53}
[[20, 2, 7, 6, 8, 11, 7], [6, 4, 18, 2, 12, 17, 13, 8], [6, 2, 11, 5], [11, 2, 27, 2, 14, 2], [16, 2, 6, 2, 10, 5, 13, 4, 9, 5], [24, 2, 9, 5, 6, 8, 13, 2, 7], [6, 2, 14, 2, 41, 2, 6, 2, 21, 5], [7, 6, 30, 17, 6, 2, 7, 26, 8], [14, 4, 21, 2, 15, 5, 23, 4, 13, 2, 7]]


In [237]:
# New text data for testing
new_texts = ["مرحبا كيف حالك", "السلام عليكم"]

# Tokenize the new text data at the character level
sequences_new = char_tokenizer.texts_to_sequences(new_texts)

# Print the results
for text, sequence in zip(new_texts, sequences_new):
    print(f"Original Text: {text}")
    print(f"Tokenized Sequence: {sequence}\n")

Original Text: مرحبا كيف حالك
Tokenized Sequence: [9, 15, 24, 14, 7, 3, 23, 10, 19, 3, 24, 7, 6, 23]

Original Text: السلام عليكم
Tokenized Sequence: [7, 6, 25, 6, 7, 9, 3, 16, 6, 10, 23, 9]



Addding Padding to the tokenizers

In [238]:

# Assuming word_sequences and char_sequences are the output of the tokenizers
word_sequences = words_tokenizer.texts_to_sequences(sentences)
char_sequences = char_tokenizer.texts_to_sequences(sentences_new)

# Add padding
word_sequences_padded = pad_sequences(word_sequences, padding='post')
char_sequences_padded = pad_sequences(char_sequences, padding='post')

In [239]:
sequence_lengths = [len(seq) for seq in word_sequences_padded]
print(len(char_sequences[5]))
print(len(char_sequences_padded[5]))

164
4263


In [240]:
# Save the tokenized sequences
with open('word_sequences.pkl', 'wb') as file:
    pickle.dump(word_sequences_padded, file)

with open('char_sequences.pkl', 'wb') as file:
    pickle.dump(char_sequences_padded, file)

In [241]:
#Testing if pickle works
with open('word_sequences.pkl', 'rb') as file:
    sequences_padded = pickle.load(file)

print(sequences_padded[0])
print(word_sequences_padded[0])

[    8     4   273    91   189    47    14   901    37   458     8  1184
  3641 41299    39   249  4154   475    12   163  1738     3   158  1175
 27872  6099 41300  5827 41301    37   458    63    37  2652     4  1362
 24279    43 17883 57670 41302  2961     2  1430  2148  9577 57671  1189
    18 21681    86    63    94  3359     6 16422  1190  1185   108    14
    94    43 27873    28   520  8288   415   264    51 57672     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [242]:
print(sequences_padded[3])
print(word_sequences_padded[3])

[   32     7  1380  1110    88    21   100     6  2543   180     3    39
  7575   798     6   179   147    18     2 16423     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [243]:
with open('char_sequences.pkl', 'rb') as file:
    char_sequences = pickle.load(file)

print(char_index.get("ُ"))
w = char_sequences[1]
print(char_tokenizer.sequences_to_texts(["19 2 3 4 5 6 7 8"]))

8
['UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK']


In [244]:
print(char_index)

{'UNK': 1, 'َ': 2, ' ': 3, 'ِ': 4, 'ْ': 5, 'ل': 6, 'ا': 7, 'ُ': 8, 'م': 9, 'ي': 10, 'و': 11, 'ن': 12, 'ه': 13, 'ب': 14, 'ر': 15, 'ع': 16, 'd': 17, 'أ': 18, 'ف': 19, 'ق': 20, 'ت': 21, 'د': 22, 'ك': 23, 'ح': 24, 'س': 25, 'ة': 26, 'ج': 27, 'إ': 28, 'ذ': 29, 'ص': 30, 'ٍ': 31, '،': 32, 'ى': 33, 'خ': 34, 'ث': 35, 'f': 36, 'ش': 37, 'ض': 38, 'ً': 39, 'ٌ': 40, 'ط': 41, 'ز': 42, 'e': 43, 'غ': 44, 'ء': 45, 'ئ': 46, 'ظ': 47, 'آ': 48, 'ؤ': 49, 'ّ': 50, 'c': 51, 'b': 52, 'a': 53}


TESTING labels


In [245]:
# sentences_with_tashkeel
tashkeel_list = []
for sentence in sentences_with_tashkeel:
    text, txt_list, harakat_list = util.extract_haraqat(sentence) 
    for i in range(len(harakat_list)):
        if len(harakat_list[i]) == 2:
            if '\u0651\u064B' in harakat_list[i]:
                harakat_list[i] = 'a'
            if '\u0651\u064C' in harakat_list[i]:
                harakat_list[i] = 'b'
            if '\u0651\u064D' in harakat_list[i]:
                harakat_list[i] = 'c'
            if '\u0651\u064E' in harakat_list[i]:
                harakat_list[i] = 'd'
            if '\u0651\u064F' in harakat_list[i]:
                harakat_list[i] = 'e'
            if '\u0651\u0650' in harakat_list[i]:
                harakat_list[i] = 'f'

    tashkeel_list.append(harakat_list)



In [246]:
with open('tashkeel_list.pkl', 'wb') as file:
    pickle.dump(tashkeel_list, file)

In [247]:
print(tashkeel_list[0:3])

[['َ', 'ْ', 'ُ', 'ُ', '', 'َ', 'ْ', '', 'َ', 'َ', 'َ', '', '', 'ْ', 'َ', 'd', 'ُ', '', 'َ', 'َ', 'ُ', '', '', 'َ', 'ْ', '', 'َ', '', 'َ', '', '', '', 'd', 'ْ', 'َ', 'ِ', 'e', '', '', 'ْ', 'ُ', '', 'َ', 'َ', 'َ', 'َ', '', 'َ', 'ْ', 'ُ', 'ُ', '', 'ِ', 'َ', 'ْ', 'ٍ', '', 'َ', 'ْ', 'َ', 'ِ', '', '', '', 'َ', 'ِ', 'ْ', 'َ', '', 'ِ', '', 'َ', 'ْ', 'ِ', '', 'َ', 'ِ', '', 'ٍ', '', 'ِ', '', 'ْ', 'ِ', 'ْ', 'َ', '', 'ِ', '', 'ُ', 'ُ', '', 'َ', '', 'َ', '', '', 'ُ', 'ِ', 'َ', '', 'ُ', 'ُ', '', 'ُ', 'ُ', '', 'ِ', 'ْ', '', '', '', 'f', '', 'ِ', '', 'َ', 'ُ', '', 'َ', 'ً', '', 'َ', 'ِ', 'ْ', 'َ', '', 'ِ', '', 'ُ', 'ْ', 'َ', 'ٍ', '', 'ِ', 'َ', 'َ', 'ٍ', '', 'َ', 'َ', 'f', '', 'ُ', 'd', '', 'ٍ', '', '', 'ْ', 'ُ', '', 'َ', 'َ', 'َ', 'َ', '', 'َ', 'ْ', 'ُ', '', '', 'ْ', 'ِ', '', 'َ', '', 'ٍ', '', 'َ', 'ْ', '', 'ِ', 'ِ', 'ْ', 'ٍ', '', 'َ', 'َ', 'َ', 'd', 'ُ', 'ُ', '', 'ُ', 'َ', '', 'َ', 'ُ', 'ْ', 'ِ', '', '', '', 'e', 'd', '', 'ِ', '', 'َ', 'ِ', 'ْ', 'َ', '', 'ِ', '', '', 'ْ', 'ُ', 'ْ', 'َ', 'ِ', '', 'ِ',

In [248]:
print(char_tokenizer.texts_to_sequences(tashkeel_list[0:1]))

[[2, 5, 8, 8, 1, 2, 5, 1, 2, 2, 2, 1, 1, 5, 2, 17, 8, 1, 2, 2, 8, 1, 1, 2, 5, 1, 2, 1, 2, 1, 1, 1, 17, 5, 2, 4, 43, 1, 1, 5, 8, 1, 2, 2, 2, 2, 1, 2, 5, 8, 8, 1, 4, 2, 5, 31, 1, 2, 5, 2, 4, 1, 1, 1, 2, 4, 5, 2, 1, 4, 1, 2, 5, 4, 1, 2, 4, 1, 31, 1, 4, 1, 5, 4, 5, 2, 1, 4, 1, 8, 8, 1, 2, 1, 2, 1, 1, 8, 4, 2, 1, 8, 8, 1, 8, 8, 1, 4, 5, 1, 1, 1, 36, 1, 4, 1, 2, 8, 1, 2, 39, 1, 2, 4, 5, 2, 1, 4, 1, 8, 5, 2, 31, 1, 4, 2, 2, 31, 1, 2, 2, 36, 1, 8, 17, 1, 31, 1, 1, 5, 8, 1, 2, 2, 2, 2, 1, 2, 5, 8, 1, 1, 5, 4, 1, 2, 1, 31, 1, 2, 5, 1, 4, 4, 5, 31, 1, 2, 2, 2, 17, 8, 8, 1, 8, 2, 1, 2, 8, 5, 4, 1, 1, 1, 43, 17, 1, 4, 1, 2, 4, 5, 2, 1, 4, 1, 1, 5, 8, 5, 2, 4, 1, 4, 1, 1, 2, 4, 1, 4, 1, 1, 1, 17, 2, 1, 2, 4, 1, 2, 1, 1, 43, 8, 1, 4, 1, 4, 1, 17, 2, 4, 1, 2, 2, 5, 4, 1, 2, 4, 2, 1, 2, 4, 5, 31, 1, 8, 2, 17, 40, 1, 2, 5, 8, 1, 2, 1, 4, 31, 1, 2, 2, 5, 2, 1, 4, 4, 1, 2, 17, 1, 1, 1, 17, 1, 4, 2, 1, 2, 1, 4, 40, 1, 4, 2, 1, 17, 4, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 4, 40, 1, 8, 2, 1, 2, 1, 1, 36, 5,